# FINN - Analysis Passes
--------------------------------------
This notebook is about analysis passes in FINN. The procedure for creating an analysis pass is shown using an example.

We'll use the following utility functions to print the source code for function calls (`showSrc()`) and to visualize a network using netron (`showInNetron()`) in the Jupyter notebook:

In [1]:
from finn.util.visualization import showSrc, showInNetron

## General Information
------------------------------
* traverses the graph structure and produces information about certain properties
* input: ModelWrapper
* returns dictionary of named properties that the analysis extracts

### Example - Quantity analysis of operation types
As an example, an analysis is designed that returns the number of nodes of the same operation types.

First the model is shown to illustrate the analysis. For this netron is used. Netron is a visualizer for neural network, deep learning and machine learning models.

In [2]:
import os
notebook_dir = os.environ['FINN_ROOT'] + "/notebooks"
showInNetron(notebook_dir + "/LFCW1A1.onnx")

Serving '/home/rohitmathurs/mtp/finn/notebooks/LFCW1A1.onnx' at http://0.0.0.0:8081


The onnx model has to be converted to a format that can be processed by FINN. This is done with ModelWrapper. As described in the short introduction, this is the format an analysis pass takes as input.

In [3]:
from qonnx.core.modelwrapper import ModelWrapper
model = ModelWrapper(notebook_dir + "/LFCW1A1.onnx")

The idea is to count all nodes that have the same operation type. The result should contain the operation types and the corresponding number of nodes that occur in the model. In the beginning an empty dictionary is created which is filled by the function and returned as result to the user at the end of the analysis.

In [4]:
def count_equal_nodes(model):
    count_dict = {}
    for node in model.graph.node:
        if node.op_type in count_dict:
            count_dict[node.op_type] +=1
        else:
            count_dict[node.op_type] = 1
    return count_dict

The function takes the model as input and iterates over the nodes. Then it is checked whether there is already an entry for the operation type in the dictionary. If this is not the case, an entry is created and set to `1`. If there is already an entry, it is incremented. If all nodes in the model have been iterated, the filled dictionary is returned.

The analysis function of ModelWrapper is used to perform the analysis just designed. It is shown below and takes the function as input and performs it by passing the model to the function.

In [5]:
showSrc(ModelWrapper.analysis)

    def analysis(self, analysis_fxn):
        """Runs given anaylsis_fxn on this model and return resulting dict."""
        return analysis_fxn(self)



The result can now simply be determined by calling the `.analysis` function.

In [6]:
print(model.analysis(count_equal_nodes))

{'Shape': 1, 'Gather': 1, 'Unsqueeze': 5, 'Concat': 1, 'Reshape': 1, 'Mul': 5, 'Sub': 1, 'Sign': 4, 'MatMul': 4, 'BatchNormalization': 3, 'Squeeze': 3}
